In [14]:
import torch
import os
import io
import sys
import csv
import six
# from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import warnings ; warnings.filterwarnings('ignore') 
import itertools
import collections
import argparse

from torchtext.vocab import Vectors, Vocab
import gensim
from collections import defaultdict, OrderedDict

from model import IQN
from trainer import Trainer
from utils import MyDataset
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline

import torch
import torch.nn as nn

In [2]:
parser = argparse.ArgumentParser(description=None)
parser.add_argument('-e', '--env', default='PongNoFrameskip-v4', type=str, help='gym environment')
parser.add_argument('-d', '--density', default=1, type=int, help='density of grid of gaussian blurs')
parser.add_argument('-r', '--radius', default=5, type=int, help='radius of gaussian blur')
parser.add_argument('-f', '--num_frames', default=100, type=int, help='number of frames in movie')
parser.add_argument('-i', '--first_frame', default=150, type=int, help='index of first frame')
parser.add_argument('-dpi', '--resolution', default=75, type=int, help='resolution (dpi)')
parser.add_argument('-s', '--save_dir', default='./movies/', type=str,
                    help='dir to save agent logs and checkpoints')
parser.add_argument('-p', '--prefix', default='default', type=str, help='prefix to help make video name unique')
parser.add_argument('-c', '--checkpoint', default='*.tar', type=str,
                    help='checkpoint name (in case there is more than one')
parser.add_argument('-o', '--overfit_mode', default=False, type=bool,
                    help='analyze an overfit environment (see paper)')

# text parameter
parser.add_argument('--max_length', type=int, default=64)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--target_update_freq', type=int, default=100)
parser.add_argument('--evaluation_freq', type=int, default=10)
parser.add_argument('--network_save_freq', type=int, default=100)
parser.add_argument('--num_actions', type=int, default=1)

parser.add_argument('--min_freq', type=int, default=10)
parser.add_argument('--embedding_dim', type=int, default=300)
parser.add_argument('--n_filters', type=int, default=50)
parser.add_argument('--filter_sizes', type=list, default=[3, 4, 5])
parser.add_argument('--pad_idx', type=list, default=1)
parser.add_argument('--gamma', type=float, default=0.0)
parser.add_argument('--learning_rate', type=float, default=2.5e-5)
parser.add_argument('--round', type=float, default=0)

parser.add_argument('--num_quantile', type=int, default=32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
parser.add_argument('--device', type=str, default=device)

args = parser.parse_args(args=[])

args.rnn = False

In [3]:
japanese_vectors = Vectors(name='../data/news/cc.ja.300.vec')

# Create Dataset
train_ds = MyDataset(
    path=os.path.join('..', 'data', 'news', 'text_train.tsv'),
    specials=['<company>', '<organization>', '<person>', '<location>'],
    max_len=args.max_length,
    vectors=japanese_vectors,
    min_freq=args.min_freq,
    phase='train'
)

test_ds = MyDataset(
    path=os.path.join('..', 'data', 'news', 'text_test.tsv'),
    max_len=args.max_length,
    vocab=train_ds.vocab,
    min_freq=args.min_freq,
    phase='val'
)

train_dl = torch.utils.data.DataLoader(
    train_ds, 
    batch_size = 1, 
    shuffle = False
)

test_dl = torch.utils.data.DataLoader(
    test_ds, 
    batch_size = 1,
    shuffle = False
)

In [16]:
train_ds[0]['State']

tensor([   2,   34,   50, 1020, 1008,   73,    4,   71,   10,    8,   26, 1004,
          37,   36,  837,   12,  103,  112,   12,   10,    8,    4,   16, 1004,
          11,   19,   12,   10,  225,  184,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])

In [11]:
train_ds.padded_list['State'][0:10]

[['<cls>',
  '決算',
  '星取表',
  '<company>',
  '上方修正',
  '●',
  'ｻﾌﾟﾗｲｽﾞﾚｼｵ',
  'N',
  '0年0月期',
  '<span>',
  '<company>',
  '0月期',
  '連結',
  '最終',
  '益',
  '0億',
  '円',
  '前年同期',
  '0億',
  '円',
  '<span>',
  '<company>',
  '0月',
  '純利益',
  '0%',
  '増',
  '0億',
  '円',
  '国内',
  '回復',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['<cls>',
  '基礎的',
  '収支',
  '比率',
  '目標',
  '守る',
  '<location>',
  '財務相',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
 

In [12]:
text_vectors = train_ds.vocab.vectors

In [23]:
embedding = nn.Embedding.from_pretrained(
            embeddings=text_vectors, freeze=False)

In [24]:
embedding(train_ds[0]['State'])[0:5]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1825,  0.0752,  0.4106,  ...,  0.2485, -0.0825, -0.2306],
        [-0.1429, -0.0541,  0.2998,  ...,  0.2327,  0.1382, -0.2645],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1292, -0.2077,  0.9288,  ...,  0.4230,  0.0498,  0.1162]],
       grad_fn=<SliceBackward>)